# Initialization

Test notebook for the C-MAPPS benchmark. Test different MLP architectures. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
import CMAPSAuxFunctions

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


# Define architectures

Define each one of the different architectures to be tested.

In [2]:
K.clear_session()  #Clear the previous tensorflow graph

lambda_regularization = 0.20

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

In [3]:
def get_compiled_model(model_def, shape, model_type='ann'):

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = model_def(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        """
        nFeatures = len(selected_features)
        shapeLSTM = (window_size, nFeatures)
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)"""
    else:
        pass

    return model

In [4]:
#Define the usable models for this notebook

models = {'shallow-20':RULmodel_SN_5}
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Process Data

In [5]:
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 128

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, 
                                  window_size, window_stride)

# Build the model

In [6]:
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = get_compiled_model(models['shallow-20'], shapeSN, model_type='ann')

tModel = SequenceTunableModelRegression('ModelRUL_SN_5', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)
tModel.data_scaler = min_max_scaler

# Load Data

In [7]:
#tModel.data_handler.data_scaler = min_max_scaler

tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 128

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ... -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ... -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ... -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ... -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ... -0.45454545  0.05511811
   0.17880983]]
[128. 128. 128. 128. 128.]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ... -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ... -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...  0.09090909  0.24409449
   0.07882403]
 [-0

# Test on dataset 1

In [13]:
iterations = 1
tModel.epochs = 200
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
num_features = len(selected_features)

windowSize = 30
windowStride = 1
constRul = 128

file = open("ResultsDataset1.csv", "w")
csvfile = csv.writer(file, lineterminator='\n')

for key, model_def in models.items():
    
    print("For model "+str(key))
    #file.write("For model "+str(key)+'\n\n')
    
    for i in range(1,2):

        dataset = i
        print("Computing for dataset "+str(i))
        #file.write("Computing for dataset "+str(i)+'\n\n')
        
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
        
        input_shape = windowSize*num_features #For simple ANN
        
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = windowSize
        tModel.data_handler.sequence_stride = windowStride
        tModel.data_handler.max_rul = constRul
        tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
        #tModel.print_data()

        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            model = get_compiled_model(model_def, input_shape, model_type='ann')

            tModel.change_model(key, model, 'keras')
            tModel.train_model(learningRate_scheduler=lrate, verbose=1)
            tModel.evaluate_model(['rhs', 'rmse'], round=2)
            #print("scores")
            
            #print(j)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
            
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time

        print("Results for model " + key)
    
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
            
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
        
file.close()

For model shallow-20
Computing for dataset 1
Loading data from file and computing dataframes
Epoch 1/200
17731/17731 [==============================] - 0s 16us/step - loss: 7703.6510 - mean_squared_error: 7695.7066
Epoch 2/200
17731/17731 [==============================] - 0s 6us/step - loss: 5725.8818 - mean_squared_error: 5715.1671
Epoch 3/200
17731/17731 [==============================] - 0s 7us/step - loss: 3877.7723 - mean_squared_error: 3861.7496
Epoch 4/200
17731/17731 [==============================] - 0s 7us/step - loss: 2614.6875 - mean_squared_error: 2592.8046
Epoch 5/200
17731/17731 [==============================] - 0s 7us/step - loss: 1930.1202 - mean_squared_error: 1903.7575
Epoch 6/200
17731/17731 [==============================] - 0s 7us/step - loss: 1569.9895 - mean_squared_error: 1541.3146
Epoch 7/200
17731/17731 [==============================] - 0s 7us/step - loss: 1309.1802 - mean_squared_error: 1278.9986
Epoch 8/200
17731/17731 [==============================] - 

17731/17731 [==============================] - 0s 7us/step - loss: 283.5471 - mean_squared_error: 235.2187
Epoch 69/200
17731/17731 [==============================] - 0s 7us/step - loss: 282.5612 - mean_squared_error: 234.5928
Epoch 70/200
17731/17731 [==============================] - 0s 7us/step - loss: 281.8623 - mean_squared_error: 234.1369
Epoch 71/200
17731/17731 [==============================] - 0s 7us/step - loss: 281.2386 - mean_squared_error: 233.7660
Epoch 72/200
17731/17731 [==============================] - 0s 7us/step - loss: 280.5188 - mean_squared_error: 233.2997
Epoch 73/200
17731/17731 [==============================] - 0s 6us/step - loss: 279.6368 - mean_squared_error: 232.6784
Epoch 74/200
17731/17731 [==============================] - 0s 6us/step - loss: 279.1088 - mean_squared_error: 232.5035
Epoch 75/200
17731/17731 [==============================] - 0s 6us/step - loss: 278.4365 - mean_squared_error: 232.0562
Epoch 76/200
17731/17731 [===========================

17731/17731 [==============================] - 0s 6us/step - loss: 260.0553 - mean_squared_error: 220.3626
Epoch 137/200
17731/17731 [==============================] - 0s 7us/step - loss: 259.9934 - mean_squared_error: 220.3200
Epoch 138/200
17731/17731 [==============================] - 0s 7us/step - loss: 259.9347 - mean_squared_error: 220.2955
Epoch 139/200
17731/17731 [==============================] - 0s 7us/step - loss: 259.7987 - mean_squared_error: 220.1767
Epoch 140/200
17731/17731 [==============================] - 0s 6us/step - loss: 259.7677 - mean_squared_error: 220.1840
Epoch 141/200
17731/17731 [==============================] - 0s 6us/step - loss: 259.6314 - mean_squared_error: 220.0607
Epoch 142/200
17731/17731 [==============================] - 0s 7us/step - loss: 259.6351 - mean_squared_error: 220.0920
Epoch 143/200
17731/17731 [==============================] - 0s 6us/step - loss: 259.5592 - mean_squared_error: 220.0465
Epoch 144/200
17731/17731 [===================

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\numpy\core\fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:125: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


# Test on all Datasets

In [7]:
datasets = [1,2,3,4]
iterations = 10
tModel.epochs = 200
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
scores ={1:[], 2:[], 3:[], 4:[]}
window_sizes = {1:30,2:20,3:30,4:18}
strides = {1:1,2:2,3:1,4:2}
max_ruls = {1:128, 2:134, 3:128, 4:134}
num_features = len(selected_features)

input_shape = None

#For each model
for key, model_def in models.items():
    file = open("results/MLP/ResultsDatasets_ScalerAtEnd"+key+".csv", "w")
    csvfile = csv.writer(file, lineterminator='\n')
    
    print("Generating statistics for model " + key)

    #For each dataset
    for i in range(1,5):
        
        print("Working on dataset " + str(i))
        
        tempScoresRMSE = np.zeros((iterations,1))
        tempScoresRHS = np.zeros((iterations,1))
        tempTime = np.zeros((iterations,1))
        
        input_shape = window_sizes[i]*num_features #For simple ANN
        #input_shape = (window_sizes,num_features) #For RNN
        
        tModel.data_handler.change_dataset(i)
        tModel.data_handler.sequence_length = window_sizes[i]
        tModel.data_handler.sequence_stride = strides[i]
        tModel.data_handler.max_rul = max_ruls[i]
        tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
        #tModel.print_data()
        
        #tModel.print_data()
        
        for j in range(iterations):

            #Model needs to be recompiled everytime since they are different runs so weights should be reinit
            model = get_compiled_model(model_def, input_shape, model_type='ann')

            tModel.change_model(key, model, 'keras')
            tModel.train_model(learningRate_scheduler=lrate, verbose=0)
            tModel.evaluate_model(['rhs', 'rmse'], round=2)
            #print("scores")
            
            #print(j)

            cScores = tModel.scores
            rmse = math.sqrt(cScores['score_1'])
            rmse2 = cScores['rmse']
            rhs = cScores['rhs']
            time = tModel.train_time
            
            tempScoresRMSE[j] = rmse2
            tempScoresRHS[j] = rhs
            tempTime[j] = time
            
        print("Results for model " + key)
    
        print(stats.describe(tempScoresRMSE))
        print(stats.describe(tempScoresRHS))
        print(stats.describe(tempTime))
            
        tempScoresRMSE = np.reshape(tempScoresRMSE, (iterations,))
        tempScoresRHS = np.reshape(tempScoresRHS, (iterations,))
        tempTime = np.reshape(tempTime, (iterations,))
        csvfile.writerow(tempScoresRMSE)
        csvfile.writerow(tempScoresRHS)
        csvfile.writerow(tempTime)
    
    file.close()

Generating statistics for model shallow-20
Working on dataset 1
Loading data from file and computing dataframes
100/100 [==============================] - 0s 728us/step
Results for model shallow-20
DescribeResult(nobs=10, minmax=(array([16.02810032]), array([16.14310999])), mean=array([16.08127321]), variance=array([0.0018356]), skewness=array([0.2206925]), kurtosis=array([-1.42694699]))
DescribeResult(nobs=10, minmax=(array([4.3769073]), array([4.58968393])), mean=array([4.44239953]), variance=array([0.00399375]), skewness=array([1.26123967]), kurtosis=array([0.95910528]))
DescribeResult(nobs=10, minmax=(array([23.04694601]), array([23.89545351])), mean=array([23.54486307]), variance=array([0.08080319]), skewness=array([-0.33239359]), kurtosis=array([-0.9684114]))
Working on dataset 2
Loading data from file and computing dataframes
259/259 [==============================] - 0s 427us/step


KeyboardInterrupt: 